# Chapter 9. Decision Analysis

[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap09.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
include("./pmfAndCdf.jl")
include("./simplestat.jl")

## The Price is Right Problem

*The Price is Right* - a gameshow. The objective is to guess the price of a collection of prizes.
The contestant who comes closest to the actual price, without going over, wins the prizes.

One of the episodes:
N Prize: dishwasher, wine cabinet, laptop, car.
L Prize: pinball machine, video arcade game, pool table, cruise of the Bahamas

Bids:
N: $26'000 (real price: $25'347, diff: $653)
L: $21'500 (real price: $21'578, diff: $78)

L wins her showcase, and due to smaller diffs also N showcase.

Several questions for a Bayesian thinker.

1. Before seeing the prizes, what prior beliefs should the contestants have about the price of the showcase?
2. After seeing the prizes, how should the contestants update those beliefs?
3. Based on the posterior distribution, what should the contestants bid?

Problem inspired by Cameron Davidson-Pilon's [book](https://dataorigami.net/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/).